In [ ]:
import numpy as np
import pandas as pd
import urllib.request, urllib.parse
from bs4 import BeautifulSoup

Import netflix shows dataset

In [ ]:
df = pd.read_csv("data/netflix_shows.csv", index_col='show_id')
df.head()

,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
show_id,,,,,,,,,,,
81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
80117401,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
70234439,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autob..."
80058654,TV Show,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,"September 8, 2018",2016,TV-Y7,1 Season,Kids' TV,When a prison ship crash unleashes hundreds of...
80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,"September 8, 2017",2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...


Separate duration into two columns, duration for movie and number of seasons for TV Show

In [ ]:
seasons = []
movie_duration = []
for i in range(len(df)):
    tmp = df.iloc[i]['duration']
    number = tmp.split(" ")[0]
    if 'Season' in tmp:
        seasons.append(number)
        movie_duration.append(np.nan)
    else:
        seasons.append(np.nan)
        movie_duration.append(number)

In [ ]:
df['seasons']=seasons
df['duration']=movie_duration

In [ ]:
df.head()

,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,seasons
show_id,,,,,,,,,,,,
81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...,NaN
80117401,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...,NaN
70234439,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,"September 8, 2018",2013,TV-Y7-FV,NaN,Kids' TV,"With the help of three human allies, the Autob...",1
80058654,TV Show,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,"September 8, 2018",2016,TV-Y7,NaN,Kids' TV,When a prison ship crash unleashes hundreds of...,1
80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,"September 8, 2017",2017,TV-14,99,Comedies,When nerdy high schooler Dani finally attracts...,NaN


### Data scrapping

scrapping IMDB rating for every movie using the HTML parser beautiful soup.

We first get the IMDB ID of the netflix show attributes, then visit IMDB.com/title/ID to get the score.

In [ ]:
def scrap_ID(title,year,media_type):
    encoding = 'utf-8'
    find_url = 'https://www.imdb.com/find' 
    values = {'q':title}
    query = urllib.parse.urlencode(values).encode(encoding)
    req = urllib.request.Request(find_url, query)
    resp = urllib.request.urlopen(req)
    html = BeautifulSoup(resp,"html")
    search = html.findAll("td", {"class": "result_text"})[:3]
    if len(search) > 0:
        for item in search: 
            if  str(item.text).find(str(year)) > 0:
                if str(media_type) == "TV Show":
                    if str(item.text).find("Series") > 0:
                        title_names = str(item.text)
                        title_IDs = str(item.a["href"])
                        break
                    else:
                        title_names = str(item.text)
                        title_IDs = str(item.a["href"])
                        break
                else:
                    title_names = str(item.text)
                    title_IDs = str(item.a["href"])
                    break

            else:
                if str(media_type) == "TV Show":
                    if str(item.text).find("Series") > 0:
                        title_names = str(item.text)
                        title_IDs = str(item.a["href"])
                        break
                    else:
                        title_names = str(item.text)
                        title_IDs = str(item.a["href"])
                        break
                else:
                    title_IDs = ''
                    break
    else:
        title_IDs = ''

    return title_IDs

In [ ]:
def scrap_rating(title_ID):
    imdb_url = "https://www.imdb.com"
    if title_ID == '':
        rating = ''
    else:
        target_url = imdb_url + title_ID
        target_url = target_url
        resp = urllib.request.urlopen(target_url)
        html = BeautifulSoup(resp,"html")
        search = html.findAll("div", {"class": "ratingValue"})
        if len(search) > 0:
            for item in search:
                rating = str(item.strong['title'])
        else:
            rating = ''

    return rating

We use fibonacci great property of taking time to build up while scalling exponentially.

In [ ]:
IMDB_rating = []
IMDB_titleID = []
i,j,c = 1,1,1
for index, row in df.iterrows():
    show_ID = scrap_ID(row.title, row.release_year, row.type)
    IMDB_titleID.append(show_ID)
    rating = scrap_rating(show_ID)
    IMDB_rating.append(rating)
    c+=1
    if c == i+j:
        print('scrapped {} IMDB rating {} left'.format(c, len(df)-c))
        i, j = j, i+j
print('done!')

scrapped 2 IMDB rating 6232 left
scrapped 3 IMDB rating 6231 left
scrapped 5 IMDB rating 6229 left
scrapped 8 IMDB rating 6226 left
scrapped 13 IMDB rating 6221 left
scrapped 21 IMDB rating 6213 left
scrapped 34 IMDB rating 6200 left
scrapped 55 IMDB rating 6179 left
scrapped 89 IMDB rating 6145 left
scrapped 144 IMDB rating 6090 left
scrapped 233 IMDB rating 6001 left
scrapped 377 IMDB rating 5857 left
scrapped 610 IMDB rating 5624 left
scrapped 987 IMDB rating 5247 left
scrapped 1597 IMDB rating 4637 left
scrapped 2584 IMDB rating 3650 left
scrapped 4181 IMDB rating 2053 left
done!


In [ ]:
IMDB_rating[:30]

['3.3 based on 307 user ratings',
 '4.8 based on 23 user ratings',
 '7.9 based on 5,406 user ratings',
 '6.0 based on 839 user ratings',
 '5.2 based on 5,570 user ratings',
 '7.0 based on 227 user ratings',
 '6.1 based on 54,262 user ratings',
 '4.7 based on 27 user ratings',
 '6.6 based on 320 user ratings',
 '5.5 based on 15,518 user ratings',
 '',
 '6.1 based on 19,531 user ratings',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '6.1 based on 47,981 user ratings',
 '5.6 based on 1,276 user ratings',
 '6.1 based on 8,537 user ratings',
 '6.2 based on 470 user ratings',
 '6.8 based on 51,616 user ratings',
 '4.7 based on 5,064 user ratings',
 '5.6 based on 11,583 user ratings',
 '5.5 based on 8 user ratings',
 '',
 '6.8 based on 337 user ratings',
 '6.3 based on 1,971 user ratings']

We can now add a rating and popularity column to our database

In [ ]:
score = []
popularity = []
for rating in IMDB_rating:
    if rating == '':
        score.append('')
        popularity.append('')
    else:
        tmp = rating.split(' ')
        score.append(tmp[0])
        popularity.append(tmp[3].replace(',',''))

In [ ]:
df.rename(columns={'rating':'PG','listed_in':'genres'}, inplace=True)
df['rating'] = score
df['popularity'] = popularity
df['IMDB_id'] = ['' if tmp == '' else tmp[7:16] for tmp in IMDB_titleID]

In [ ]:
df.head(5)

,type,title,director,cast,country,date_added,release_year,PG,duration,genres,description,seasons,rating,popularity,IMDB_id
show_id,,,,,,,,,,,,,,,
81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...,NaN,3.3,307,tt9428190
80117401,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...,NaN,4.8,23,tt6999080
70234439,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,"September 8, 2018",2013,TV-Y7-FV,NaN,Kids' TV,"With the help of three human allies, the Autob...",1,7.9,5406,tt1659175
80058654,TV Show,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,"September 8, 2018",2016,TV-Y7,NaN,Kids' TV,When a prison ship crash unleashes hundreds of...,1,6.0,839,tt3604232
80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,"September 8, 2017",2017,TV-14,99,Comedies,When nerdy high schooler Dani finally attracts...,NaN,5.2,5570,tt6119504


reorder for readability

In [ ]:
df = df[['IMDB_id', 'type', 'title', 'release_year', 'rating', 'popularity', 'genres', 'PG', 'duration', 'seasons', 'director','date_added', 'country', 'cast', 'description']]

In [ ]:
df.head(10)

,IMDB_id,type,title,release_year,rating,popularity,genres,PG,duration,seasons,director,date_added,country,cast,description
show_id,,,,,,,,,,,,,,,
81145628,tt9428190,Movie,Norm of the North: King Sized Adventure,2019,3.3,307,"Children & Family Movies, Comedies",TV-PG,90,NaN,"Richard Finn, Tim Maltby","September 9, 2019","United States, India, South Korea, China","Alan Marriott, Andrew Toth, Brian Dobson, Cole...",Before planning an awesome wedding for his gra...
80117401,tt6999080,Movie,Jandino: Whatever it Takes,2016,4.8,23,Stand-Up Comedy,TV-MA,94,NaN,NaN,"September 9, 2016",United Kingdom,Jandino Asporaat,Jandino Asporaat riffs on the challenges of ra...
70234439,tt1659175,TV Show,Transformers Prime,2013,7.9,5406,Kids' TV,TV-Y7-FV,NaN,1,NaN,"September 8, 2018",United States,"Peter Cullen, Sumalee Montano, Frank Welker, J...","With the help of three human allies, the Autob..."
80058654,tt3604232,TV Show,Transformers: Robots in Disguise,2016,6.0,839,Kids' TV,TV-Y7,NaN,1,NaN,"September 8, 2018",United States,"Will Friedle, Darren Criss, Constance Zimmer, ...",When a prison ship crash unleashes hundreds of...
80125979,tt6119504,Movie,#realityhigh,2017,5.2,5570,Comedies,TV-14,99,NaN,Fernando Lebrija,"September 8, 2017",United States,"Nesta Cooper, Kate Walsh, John Michael Higgins...",When nerdy high schooler Dani finally attracts...
80163890,tt4418844,TV Show,Apaches,2016,7.0,227,"Crime TV Shows, International TV Shows, Spanis...",TV-MA,NaN,1,NaN,"September 8, 2017",Spain,"Alberto Ammann, Eloy Azorín, Verónica Echegui,...",A young journalist is forced into a life of cr...
70304989,tt1971325,Movie,Automata,2014,6.1,54262,"International Movies, Sci-Fi & Fantasy, Thrillers",R,110,NaN,Gabe Ibáñez,"September 8, 2017","Bulgaria, United States, Spain, Canada","Antonio Banderas, Dylan McDermott, Melanie Gri...","In a dystopian future, an insurance adjuster f..."
80164077,tt8134310,Movie,Fabrizio Copano: Solo pienso en mi,2017,4.7,27,Stand-Up Comedy,TV-MA,60,NaN,"Rodrigo Toro, Francisco Schultz","September 8, 2017",Chile,Fabrizio Copano,Fabrizio Copano takes audience participation t...
80117902,tt7293754,TV Show,Fire Chasers,2017,6.6,320,"Docuseries, Science & Nature TV",TV-MA,NaN,1,NaN,"September 8, 2017",United States,NaN,"As California's 2016 fire season rages, brave ..."


In [ ]:
df.to_csv("data/netflix_shows_with_IMDB_rating.csv", sep='|')